In [6]:
import pandas as pd 
import sqlite3
import sys
from IPython.display import display, HTML
import os
import sys
sys.path.insert(0, '../')
import util.util_newssniffer_parsing as unp
import json

In [14]:
path_to_data = '../data/diffengine-diffs/spark-output'
con = sqlite3.connect(os.path.join(path_to_data, 'wp-matched-sentences.db'))

In [15]:
pd.read_sql('''SELECT 
                    name
                FROM 
                    sqlite_master 
                WHERE 
                    type ='table' AND 
                    name NOT LIKE 'sqlite_%';
''', con=con)

,name
0,matched_sentences
1,split_sentences


In [16]:
pd.read_sql('''SELECT * FROM MATCHED_SENTENCES limit 5;''', con = con)

,entry_id,version_x,version_y,sent_idx_x,sent_idx_y,avg_sentence_distance_x,avg_sentence_distance_y
0,1265845,0,1,44.0,44.0,0.000,0.000
1,1343259,0,1,25.0,25.0,0.000,0.000
2,1317302,1,2,3.0,3.0,0.000,0.000
3,1361393,0,1,37.0,37.0,0.000,0.000
4,1291614,0,1,46.0,45.0,0.125,0.125


In [17]:
pd.read_sql('''SELECT * FROM SPLIT_SENTENCES limit 5;''', con = con)

,entry_id,version,sent_idx,sentence
0,1094962,0,0,BEIRUT — The Latest developments on the war in...
1,1094962,0,1,(all times local): 11:45 a.m.
2,1094962,0,2,The Russian defense ministry says its airstrik...
3,1094962,0,3,The defense ministry in a statement on Thursda...
4,1094962,0,4,A Syrian government offensive around the city ...


In [18]:
# format output

In [19]:
five_docs_arcs = pd.read_sql('''
    SELECT * FROM MATCHED_SENTENCES 
    WHERE entry_id in (
        SELECT DISTINCT entry_id from MATCHED_SENTENCES
        where sent_idx_x != sent_idx_y
        limit 5
    )
    ORDER BY entry_id, version_x, version_y, sent_idx_x, sent_idx_y;
''', con = con)

In [20]:
one_doc_arcs = (five_docs_arcs
                     .loc[lambda df: df['entry_id'] == 1268988]
                     .loc[lambda df: df['version_x'] == 0]
                     .loc[lambda df: df['version_y'] == 1]
)
one_doc_arcs_dict = (one_doc_arcs
                     [['sent_idx_x', 'sent_idx_y']]
                     .to_dict(orient='rows')
                    )

In [21]:
one_doc_arcs

,entry_id,version_x,version_y,sent_idx_x,sent_idx_y,avg_sentence_distance_x,avg_sentence_distance_y
0,1268988,0,1,NaN,9.0,NaN,NaN
1,1268988,0,1,NaN,18.0,NaN,NaN
2,1268988,0,1,NaN,19.0,NaN,NaN
3,1268988,0,1,NaN,20.0,NaN,NaN
4,1268988,0,1,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...
59,1268988,0,1,38.0,50.0,0.202069,NaN
60,1268988,0,1,39.0,50.0,0.083333,0.083333
61,1268988,0,1,40.0,51.0,NaN,0.352941
62,1268988,0,1,40.0,52.0,0.147059,0.147059


In [22]:
one_doc_nodes = pd.read_sql('''
    SELECT * FROM SPLIT_SENTENCES 
    WHERE entry_id = 1268988
    AND version IN (0, 1)
    ORDER BY version, sent_idx
''', con = con)

one_doc_nodes_dict = (one_doc_nodes
                      [['version', 'sent_idx', 'sentence']]
                      .to_dict(orient='rows'))

one_doc_nodes.head(2)

,entry_id,version,sent_idx,sentence
0,1268988,0,0,Why is this man smiling?
1,1268988,0,1,President Obama’s chosen successor suffered a ...


In [23]:
pd.options.display.max_colwidth = 500

In [24]:
(one_doc_arcs[['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y', 'avg_sentence_distance_x', 'avg_sentence_distance_y']]
 .merge(
    one_doc_nodes,
#     how='outer',
    left_on=['version_x', 'sent_idx_x'],
    right_on=['version', 'sent_idx']
 )[['version_x', 'version_y', 'sent_idx_x', 'sent_idx_y', 'sentence', 'avg_sentence_distance_x', 'avg_sentence_distance_y']]
 .merge(
    one_doc_nodes,
#     how='outer',
    left_on=['version_y', 'sent_idx_y'],
    right_on=['version', 'sent_idx']
 )[['version_x', 'version_y', 'sent_idx_x', 'sent_idx_y', 'sentence_x', 'sentence_y', 'avg_sentence_distance_x', 'avg_sentence_distance_y']]
)

,version_x,version_y,sent_idx_x,sent_idx_y,sentence_x,sentence_y,avg_sentence_distance_x,avg_sentence_distance_y
0,0,1,0.0,0.0,Why is this man smiling?,Why is this man smiling?,0.000000e+00,0.000000e+00
1,0,1,1.0,1.0,President Obama’s chosen successor suffered a devastating loss last week to a man who made a primary campaign issue of Obama’s “disastrous” management of the country.,President Obama’s chosen successor suffered a devastating loss last week to a man who made a primary campaign issue of Obama’s “disastrous” management of the country.,0.000000e+00,0.000000e+00
2,0,1,2.0,2.0,"The Democratic Party is in a shambles, outnumbered in state legislatures, governors’ mansions, the House and the Senate.","The Democratic Party is in a shambles, outnumbered in state legislatures, governors’ mansions, the House and the Senate.",5.280007e-07,5.280007e-07
3,0,1,3.0,3.0,Conservative control of the Supreme Court seems likely for another generation.,Conservative control of the Supreme Court seems likely for another generation.,0.000000e+00,0.000000e+00
4,0,1,4.0,4.0,"Obama’s legacy is in tatters, as his trade policy, his foreign policy and his beloved Obamacare are set to be dismantled.","Obama’s legacy is in tatters, as his trade policy, his foreign policy and his beloved Obamacare are set to be dismantled.",4.595624e-07,4.595624e-07
5,0,1,5.0,5.0,"And yet when Obama entered the White House briefing room for a post-election news conference Monday afternoon, everything was, if not awesome, then pretty darned good.","And yet when Obama entered the White House briefing room for a post-election news conference Monday afternoon, everything was, if not awesome, then pretty darned good.",3.334983e-08,3.334983e-08
6,0,1,6.0,6.0,"“We are indisputably in a stronger position today than we were when I came in eight years ago,” he began.","“We are indisputably in a stronger position today than we were when I came in eight years ago,” he began.",0.000000e+00,0.000000e+00
7,0,1,7.0,7.0,"“Jobs have been growing for 73 straight months, incomes are rising, poverty is falling, the uninsured rate is at the lowest level on record, carbon emissions have come down without impinging on our growth…” The happy talk kept coming: “Unemployment rate is low as it has been in eight, nine years, incomes and wages have both gone up over the last year faster than they have in a decade or two… The financial systems are stable.","“Jobs have been growing for 73 straight months, incomes are rising, poverty is falling, the uninsured rate is at the lowest level on record, carbon emissions have come down without impinging on our growth .",3.449258e-01,3.465342e-01
8,0,1,8.0,14.0,The stock market is hovering around its all-time high and 401,The financial systems are stable.,NaN,3.181820e-01
9,0,1,8.0,15.0,The stock market is hovering around its all-time high and 401,The stock market is hovering around its all-time high and 401,1.818185e-01,NaN


In [25]:
one_doc_nodes

,entry_id,version,sent_idx,sentence
0,1268988,0,0,Why is this man smiling?
1,1268988,0,1,President Obama’s chosen successor suffered a devastating loss last week to a man who made a primary campaign issue of Obama’s “disastrous” management of the country.
2,1268988,0,2,"The Democratic Party is in a shambles, outnumbered in state legislatures, governors’ mansions, the House and the Senate."
3,1268988,0,3,Conservative control of the Supreme Court seems likely for another generation.
4,1268988,0,4,"Obama’s legacy is in tatters, as his trade policy, his foreign policy and his beloved Obamacare are set to be dismantled."
...,...,...,...,...
91,1268988,1,49,"“Obviously there are people out there who are feeling deeply disaffected,” the president said with his cool detachment."
92,1268988,1,50,"In an election this close — Clinton, let’s not forget, won the popular vote — any factor could have made the difference: being a candidate of the establishment in a time of change, resistance to a woman as president and backlash against the first black president, and FBI Director James B. Comey’s last-minute intervention in the election."
93,1268988,1,51,But millions of Americans are justifiably anxious about their economic well-being.
94,1268988,1,52,"And if Clinton and Obama had limited the build-on-success theme during the campaign in favor of a more populist vision and policies, they really would have something to smile about this week."


In [112]:
ls ../data/tmp

one_wp_doc_test.csv
wp-many-docs-with-par-break_sentences.csv
wp-many-docs-with-par-break_word-matching.csv
wp-many-docs-with-sent-breaks_sentences.csv
wp-many-docs-with-sent-breaks_word-matching.csv
wp-one-doc_sentences.csv
wp-one-doc_word-matching.csv


In [274]:
file = '../data/tmp/wp-one-doc_sentences.csv'
file = '../data/tmp/wp-many-docs-with-par-break_sentences.csv'
file = '../data/tmp/wp-many-docs-with-sent-breaks_sentences.csv'
doc_nodes = (pd
             .read_csv(file, index_col=0)
             .sort_values(['entry_id', 'version', 'sent_idx'])
             [['entry_id', 'version', 'sent_idx', 'sentence']]
            )
doc_nodes_dict = (doc_nodes
                      .to_dict(orient='rows'))

In [284]:
file = '../data/tmp/wp-one-doc_word-matching.csv'
file = '../data/tmp/wp-many-docs-with-par-break_word-matching.csv'
file = '../data/tmp/wp-many-docs-with-sent-breaks_word-matching.csv'
doc_arcs = (
    pd.read_csv(file, index_col=0)
     .sort_values(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])
     [['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y']]
)
doc_arcs_dict = doc_arcs.to_dict(orient='rows')

# get HTML diffs
doc_arcs = (doc_arcs
 .merge(doc_nodes, how='outer', 
              right_on=['entry_id', 'version', 'sent_idx'],
              left_on=['entry_id', 'version_x', 'sent_idx_x'] ,
  ).drop(['version', 'sent_idx'], axis=1)
 .merge(doc_nodes, how='outer', 
              right_on=['entry_id', 'version', 'sent_idx'],
              left_on=['entry_id', 'version_y', 'sent_idx_y'] ,
  ).drop(['version', 'sent_idx'], axis=1) 
)

grouped_arcs = (doc_arcs
 .groupby(['entry_id', 'version_x', 'version_y'])
 .apply(lambda df: df[['version_x', 'version_y', 'sent_idx_x', 'sent_idx_y']].to_dict(orient='rows'))
 .to_frame('arcs')
)

grouped_nodes = (doc_nodes
 .groupby(['entry_id', 'version'])
 .apply(lambda df: df[['version', 'sent_idx', 'sentence']].to_dict(orient='rows'))
).to_frame('nodes').reset_index()

matched_grouped_nodes = (grouped_nodes
 .merge(
     grouped_nodes.assign(next_vers=lambda df: df['version'] - 1), 
     left_on=['entry_id', 'version'], 
     right_on=['entry_id', 'next_vers']
 )
 .assign(nodes=lambda df: df['nodes_x'] + df['nodes_y'])
 [['entry_id', 'version_x', 'version_y', 'nodes']]
 .set_index(['entry_id', 'version_x', 'version_y'])
)

output = (
    pd.concat([matched_grouped_nodes, grouped_arcs], axis=1)
    .to_dict(orient='index')
)

output = {str(k): v for k, v in output.items()}

with open('../evaluation/data/sample_datum_many_sent_breaks.json', 'w') as f:
    json.dump(output, f )

In [ ]:
pd.options.display.min_rows = 50
(doc_arcs
 .loc[lambda df: df['entry_id'] == 728217]
 .loc[lambda df: df['version_x'] == 2]
 .loc[lambda df: df['version_y'] == 3]
)

In [ ]:
doc_nodes.loc[lambda df: df['version'] == 2]

In [159]:
import spacy

In [183]:
nlp = spacy.load('en_core_web_lg', disable=['tagger', 'ner'])

In [252]:
doc_nodes.head(2)

,entry_id,version,sent_idx,sentence
14020,728205,0,0,MOSCOW --
14021,728205,0,1,Russians love to think of their neighbors the ...


In [181]:
one_doc_orig = pd.read_sql('select * from entryversion where entry_id = 728217 AND version in (2, 3)', con=con)

In [ ]:
nlp = spacy.load('en_core_web_lg', disable=['tagger', 'ner'])

In [262]:
doc_nodes.head(1)

,entry_id,version,sent_idx,sentence
14020,728205,0,0,MOSCOW --


In [271]:
import re
def get_sentences(x, version=None, entry_id=None):
    output_sentences = []
    pars = re.split('</p>\s*<p>', one_doc_orig['summary'][0])
    s_idx = 0
    for p_idx, p in enumerate(pars):
        sents = list(map(lambda s: s.text, nlp(p).sents))
        sents = list(map(lambda s: ' '.join(s.split()), sents))
        for s in sents:
            output_sentences.append({
                'par_idx': p_idx,
                'sentence': s,
                'sent_idx': s_idx,
                'entry_id': entry_id,
                'version': version
            })
            s_idx += 1
    
    return output_sentences

In [195]:
one_doc_summ = (one_doc_orig['summary']
 .str.replace('</p>\s*<p>', ' ', regex=True)
 .str.replace('<p>', ' ').str.replace('</p>', ' ')
 [0]
)

In [217]:
import unidecode
o = unidecode.unidecode(o)

In [220]:
o = 'Congress will be key to determining whether the NSA bulk collection ends or whether it is transformed -- maybe through the use of a third party or mandated data retention by the phone companies. In the House, Rep. James F. Sensenbrenner Jr. (R-Wis.) said, "The bottom line is real reform cannot be done by presidential fiat." Congressional action "should be taken to protect Americans\' civil liberties by reining in the NSA, ending bulk collection," he said in a statement Friday. Sensenbrenner and Sen. Patrick J. Leahy (D-Vt.) are sponsoring legislation , the USA Freedom Act, in their respective chambers that would outlaw bulk collection.'

In [221]:
get_sentences(o)

['Congress will be key to determining whether the NSA bulk collection ends or whether it is transformed -- maybe through the use of a third party or mandated data retention by the phone companies.',
 'In the House, Rep. James F. Sensenbrenner Jr. (R-Wis.) said, "The bottom line is real reform cannot be done by presidential fiat.',
 '" Congressional action "should be taken to protect Americans\' civil liberties by reining in the NSA, ending bulk collection," he said in a statement Friday.',
 'Sensenbrenner and Sen. Patrick J. Leahy (D-Vt.) are sponsoring legislation , the USA Freedom Act, in their respective chambers that would outlaw bulk collection.']

In [224]:
(doc_nodes
 .loc[lambda df: df['version'] == 2]
 .loc[lambda df: df['sentence'].apply(lambda x: x in o)].iloc[:7]['sentence'].tolist()
)

['In the House, Rep.',
 'James F. Sensenbrenner Jr.',
 '(R-Wis.',
 ') said, "The bottom line is real reform cannot be done by presidential fiat." Congressional action "should be taken to protect Americans\' civil liberties by reining in the NSA, ending bulk collection," he said in a statement Friday.',
 'Sensenbrenner and Sen.',
 'Patrick J. Leahy (D-Vt.',
 '.']

In [175]:
all_summs = pd.read_sql('select summary from entryversion', con=con)

In [179]:
from tqdm.auto import tqdm

In [199]:
all_summs = (all_summs['summary']
 .str.replace('</p>\s*<p>', ' ', regex=True)
 .str.replace('<p>', ' ').str.replace('</p>', ' '))

In [ ]:
for summ in tqdm(all_summs, total=len(all_summs)):
    get_sentences(summ)

In [272]:
o = get_sentences(summ)

In [162]:
con = sqlite3.connect('../data/diffengine-diffs/db/newssniffer-washpo.db')

In [163]:
pd.read_sql('select * from entryversion limit 2', con=con)

,index,version,title,created,url,source,entry_id,archive_url,num_versions,summary,joint_key,id
0,334073,0,Ukraine enacts harsh laws against protests,2014-01-20 00:20:00+00:00,http://www.washingtonpost.com/world/europe/ukr...,washpo,728205,https://www.newssniffer.co.uk/articles/728205/...,4,MOSCOW — Russians love to think of their neigh...,728205-0,728205-0
1,334074,1,Ukraine enacts harsh laws against protests,2014-01-25 17:00:00+00:00,http://www.washingtonpost.com/world/europe/ukr...,washpo,728205,https://www.newssniffer.co.uk/articles/728205/...,4,MOSCOW — Russians love to think of their neigh...,728205-1,728205-1


In [7]:
one_doc_orig = pd.read_sql('select * from entryversion where entry_id = 1268988', con=con)

In [9]:
mkdir ../data/tmp

In [10]:
one_doc_orig.to_csv('../data/tmp/one_wp_doc_test.csv')

# Select data for Annotation

In [7]:
import sys
sys.path.insert(0, '..')
from util import util_refactorings as ur
import sqlite3
import pandas as pd 

# ! gunzip ../data/diffengine-diffs/spark-output/nyt-matched-sentences.db.gz
! ls ../data/diffengine-diffs/spark-output/nyt-matched-sentences.db

conn = sqlite3.connect('../data/diffengine-diffs/spark-output/nyt-matched-sentences.db')

pd.read_sql('''SELECT 
                    name
                FROM 
                    sqlite_master 
                WHERE 
                    type ='table' AND 
                    name NOT LIKE 'sqlite_%';
''', con=conn)

../data/diffengine-diffs/spark-output/nyt-matched-sentences.db


,name
0,matched_sentences
1,split_sentences


In [5]:
low_count_versions = pd.read_sql('''
    with c1 as 
        (SELECT entry_id, version, COUNT(1) as c from split_sentences GROUP BY entry_id, version)
    SELECT entry_id, version from c1
    WHERE c < 10 and c > 5
''', con=conn)

# get join keys
low_count_entry_ids = ', '.join(list(map(str, low_count_versions['entry_id'].unique())))
joint_keys = low_count_versions.pipe(lambda df: df['entry_id'].astype(str) + '-' + df['version'].astype(str))
joint_keys = "'%s'" % "', '".join(joint_keys.tolist())

# matched sentences
matched_sentences = pd.read_sql('''
    WITH c1 as ( 
    SELECT *, 
    entry_id || '-' || version_x as key_x,
    entry_id || '-' || version_y as key_y 
    FROM matched_sentences 
    )
    SELECT *
    FROM c1
    WHERE key_x in (%s) AND key_y  in (%s)
    ''' % (joint_keys, joint_keys)
, con=conn)

# get split sentences
split_sentences = pd.read_sql('''
    with c1 AS (
        SELECT *, entry_id || '-' || version as key FROM split_sentences
    )
    SELECT * from c1
    WHERE key IN (%s)
''' % joint_keys, con=conn)

# merge both
merged_matched_sentences = (matched_sentences
 .merge(
    split_sentences, left_on=['entry_id', 'version_x', 'sent_idx_x'], right_on=['entry_id', 'version', 'sent_idx'],
    how='left'
 ).drop(['version', 'sent_idx', 'key', 'key_x', 'key_y'], axis=1)
 .merge(
    split_sentences, left_on=['entry_id', 'version_y', 'sent_idx_y'], right_on=['entry_id', 'version', 'sent_idx'],
    how='left'
 ).drop(['version', 'sent_idx', 'key',], axis=1)
)

In [72]:
merged_matched_sentences.loc[lambda df: df['sent_idx_x'].isnull()]

,entry_id,version_x,version_y,sent_idx_x,sent_idx_y,avg_sentence_distance_x,avg_sentence_distance_y,sentence_x,sentence_y
24,1651691,0,1,NaN,6.0,NaN,NaN,NaN,There has not been a proclamation about Mr. Mc...
42,1650749,0,1,NaN,1.0,NaN,NaN,NaN,A 55-year-old woman came forward to the police...
155,1165597,0,1,NaN,3.0,NaN,NaN,NaN,There was no immediate claim of responsibility...
245,1322807,1,2,NaN,7.0,NaN,NaN,NaN,The whale stranding was the largest in the cou...
307,1598136,3,4,NaN,1.0,NaN,NaN,NaN,"In each newsletter, our gender writer, Maya Sa..."
...,...,...,...,...,...,...,...,...,...
9672,1450768,0,1,NaN,4.0,NaN,NaN,NaN,"Zhaira Franco, 35, who works for Facebook in s..."
9676,1450768,0,1,NaN,6.0,NaN,NaN,NaN,An aftershock with a magnitude of 5.7 and an e...
9698,1223039,0,1,NaN,7.0,NaN,NaN,NaN,Schlafly’s obituary will be posted soon.
9709,1223039,0,1,NaN,6.0,NaN,NaN,NaN,” A full version of Mrs.


In [71]:
doc_arcs_dict = matched_sentences.to_dict(orient='rows')

# get HTML diffs
doc_arcs = (matched_sentences
 .merge(split_sentences, how='outer', 
              right_on=['entry_id', 'version', 'sent_idx'],
              left_on=['entry_id', 'version_x', 'sent_idx_x'] ,
  ).drop(['version', 'sent_idx'], axis=1)
 .merge(split_sentences, how='outer', 
              right_on=['entry_id', 'version', 'sent_idx'],
              left_on=['entry_id', 'version_y', 'sent_idx_y'] ,
  ).drop(['version', 'sent_idx'], axis=1) 
)

grouped_arcs = (matched_sentences
 .groupby(['entry_id', 'version_x', 'version_y'])
 .apply(lambda df: df[['version_x', 'version_y', 'sent_idx_x', 'sent_idx_y']].to_dict(orient='rows'))
 .to_frame('arcs')
)

grouped_nodes = (split_sentences
 .groupby(['entry_id', 'version'])
 .apply(lambda df: df[['version', 'sent_idx', 'sentence']].to_dict(orient='rows'))
).to_frame('nodes').reset_index()

matched_grouped_nodes = (grouped_nodes
 .merge(
     grouped_nodes.assign(next_vers=lambda df: df['version'] - 1), 
     left_on=['entry_id', 'version'], 
     right_on=['entry_id', 'next_vers']
 )
 .assign(nodes=lambda df: df['nodes_x'] + df['nodes_y'])
 [['entry_id', 'version_x', 'version_y', 'nodes']]
 .set_index(['entry_id', 'version_x', 'version_y'])
)

output = (
    pd.concat([matched_grouped_nodes, grouped_arcs], axis=1)
    .to_dict(orient='index')
)

output = {str(k): v for k, v in output.items()}

import json
with open('../evaluation/data/sample_datum_small.json', 'w') as f:
    json.dump(output, f )

In [ ]:
# joined_output = pd.read_sql('''
#     with c1 as (
#         SELECT entry_id, version, COUNT(1) as c from split_sentences GROUP BY entry_id, version
#     ),
#     c2 as (
#         SELECT entry_id, version from c1
#         WHERE c < 10 and c > 5
#     ),
#     c3 as (
#         SELECT * FROM matched_sentences m
#         INNER JOIN c2 ON m.entry_id = c2.entry_id AND (m.version_x = c2.version OR m.version_y = c2.version)
#     )
#     SELECT c3.*, s1.sentence as sentence_x, s2.sentence as sentence_y
#     FROM c3
#     JOIN split_sentences as s1 ON s1.entry_id = c3.entry_id AND s1.version = c3.version_x AND s1.sent_idx = c3.sent_idx_x
#     JOIN split_sentences as s2 ON s2.entry_id = c3.entry_id AND s2.version = c3.version_y AND s2.sent_idx = c3.sent_idx_y
#     '''
# , con=conn)

In [130]:
joined_output

,entry_id,version_x,version_y,sent_idx_x,sent_idx_y,avg_sentence_distance_x,avg_sentence_distance_y
0,1344122,1,2,7.0,NaN,NaN,NaN
1,1348302,0,1,11.0,6.0,0.000000,0.000000
2,1344122,1,2,27.0,NaN,NaN,NaN
3,1344122,0,1,NaN,5.0,NaN,NaN
4,1344122,0,1,4.0,17.0,0.000000,0.000000
...,...,...,...,...,...,...,...
47573,808961,0,1,NaN,6.0,NaN,NaN
47574,799173,0,1,2.0,2.0,0.209182,0.205092
47575,799173,0,1,3.0,3.0,0.000000,0.000000
47576,803604,2,3,3.0,3.0,0.000000,0.000000


In [7]:
# We wish to study:

# for 
## Additions    -- nodes with no edge connections (node on right) +- 5 sentences
## Subtractions -- nodes with no edge connections (node on left)  +- 5 sentences
######  find all articles with < 10 sentences
###### find unconnected single nodes
###### ask users to match all nodes, say it's OK if there are nodes without matches


# Edits        -- edge connections with non-zero distance        +- 3 sentences
###### test node connections with diff range from 0 - 1
###### to generate connection strength up to one, choose short articles


# Refactors    -- edge-crossing                                  all sentences within refactor +- 3 sentences
###### choose small articles
###### test refactors with sentence length > 5                            
###### all refactors with distance <= 7A

In [8]:
# open questions: split on the sentence or the </p><p>

In [2]:
import pandas as pd 

In [3]:
file = '../data/tmp/wp-one-doc_sentences.csv'
file = '../data/tmp/wp-many-docs-with-par-break_sentences.csv'
file = '../data/tmp/wp-many-docs-with-sent-breaks_sentences.csv'
doc_nodes = (pd
             .read_csv(file, index_col=0)
             .sort_values(['entry_id', 'version', 'sent_idx'])
             [['entry_id', 'version', 'sent_idx', 'sentence']]
            )
doc_nodes_dict = (doc_nodes
                      .to_dict(orient='rows'))

file = '../data/tmp/wp-one-doc_word-matching.csv'
file = '../data/tmp/wp-many-docs-with-par-break_word-matching.csv'
file = '../data/tmp/wp-many-docs-with-sent-breaks_word-matching.csv'
doc_arcs = (
    pd.read_csv(file, index_col=0)
     .sort_values(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])
#      [['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y']]
)
doc_arcs_dict = doc_arcs.to_dict(orient='rows')

# get HTML diffs
doc_arcs = (doc_arcs
 .merge(doc_nodes, how='outer', 
              right_on=['entry_id', 'version', 'sent_idx'],
              left_on=['entry_id', 'version_x', 'sent_idx_x'] ,
  ).drop(['version', 'sent_idx'], axis=1)
 .merge(doc_nodes, how='outer', 
              right_on=['entry_id', 'version', 'sent_idx'],
              left_on=['entry_id', 'version_y', 'sent_idx_y'] ,
  ).drop(['version', 'sent_idx'], axis=1) 
)

In [8]:
edit_statistics = (doc_arcs
 .groupby(['entry_id', 'version_x', 'version_y'])
 .apply(lambda df: pd.Series({
     'mean x dist': df['avg_sentence_distance_x'].mean(),
     'mean y dist': df['avg_sentence_distance_y'].mean(),
     'num sents removed' : df['sent_idx_y'].isnull().sum(),
     'num sents added' : df['sent_idx_x'].isnull().sum(),
     'refactors': ur.find_refactors_for_doc(df[['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y']].dropna().astype(int)),
 }))
 .assign(num_refactors=lambda df: df['refactors'].str.len())
 .assign(overall_mean=lambda df: df[['mean x dist', 'mean y dist']].mean(axis=1))
)

In [50]:
edit_statistics.loc[lambda df: df['num_refactors'] > 0]

,,,mean x dist,mean y dist,num sents removed,num sents added,refactors,num_refactors,overall_mean
entry_id,version_x,version_y,,,,,,,
731517,0.0,1.0,3.137218e-09,3.137218e-09,0,0,"[(41, 4)]",1,3.137218e-09
731908,1.0,2.0,1.807711e-02,1.811120e-02,0,1,"[(40, 41), (41, 42), (42, 43), (43, 44), (44, ...",7,1.809416e-02


In [54]:
edit_statistics.sort_values(['overall_mean'],ascending=False).head(10)

,,,mean x dist,mean y dist,num sents removed,num sents added,refactors,num_refactors,overall_mean
entry_id,version_x,version_y,,,,,,,
731509,0.0,1.0,0.414310,0.414310,89,80,[],0,0.414310
728213,0.0,1.0,0.408447,0.408447,25,41,[],0,0.408447
732006,0.0,1.0,0.272851,0.271922,11,12,[],0,0.272387
731831,1.0,2.0,0.263184,0.263184,34,25,[],0,0.263184
731909,0.0,1.0,0.255684,0.255801,10,32,[],0,0.255743
731959,0.0,1.0,0.229425,0.229033,3,31,[],0,0.229229
732326,0.0,1.0,0.204439,0.204759,1,1,[],0,0.204599
730171,1.0,2.0,0.205152,0.195133,0,0,[],0,0.200143
732938,1.0,2.0,0.186434,0.185600,8,30,[],0,0.186017


In [17]:
aggregated_sent_idxs = (doc_arcs
         [['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y']].dropna().astype(int)
         .groupby(['entry_id', 'version_x', 'version_y'])[['sent_idx_x', 'sent_idx_y']]
         .aggregate(list)
)

In [19]:
from tqdm.auto import tqdm

In [37]:
refactors = []
for s_x, s_y in tqdm( aggregated_sent_idxs.itertuples(index=False), total=len(aggregated_sent_idxs)):
    r = ur.find_refactors_for_doc(sents_old=s_x, sents_new=s_y)
    refactors.append(r)

  0%|          | 0/306 [00:00<?, ?it/s]

In [38]:
list(filter(lambda x: len(x) > 0, refactors))

[[(41, 4)],
 [(40, 41), (41, 42), (42, 43), (43, 44), (44, 45), (45, 46), (46, 47)]]

In [56]:
import torch
from torch import nn

In [58]:
w = nn.Linear(100, 1, bias=False)

In [64]:
t = torch.rand(100, 100, 100)

In [66]:
w(t).shape 

torch.Size([100, 100, 1])

In [73]:
torch.vstack([torch.tensor([1,2]), torch.tensor([1,2]), torch.tensor([1,2])])

tensor([[1, 2],
        [1, 2],
        [1, 2]])

In [309]:
max(crossings.items(), key=lambda x: len(x[1]))

((3, 1), {(1, 2), (1, 3), (2, 2), (2, 3)})

In [308]:
remove_all_crossings(crossings)

remove :  (3, 1)
(1, 2)  :  set()
(1, 3)  :  set()
(2, 2)  :  {(1, 3)}
(2, 3)  :  set()

remove :  (1, 3)
(1, 2)  :  set()
(2, 2)  :  set()
(2, 3)  :  set()



1      1 
    \
2   -- 2
   /\\
3      3

In [312]:
max(map(lambda x: abs(x[1] - x[0]), [(1, 2), (1, 3), (2, 2), (2, 3)]))

2

# Find total number of short documents

In [102]:
import glob
from tqdm.auto import tqdm
import os 

In [105]:
source

'../data/diffengine-diffs/spark-output/nyt-matched-sentences.db.gz'

In [110]:
all_matched = []
all_split = []
for source in tqdm(glob.glob('../data/diffengine-diffs/spark-output/*')):
    name = source.split('/')[-1].split('-')[0]
    if os.path.exists(source):
        print('unzipping %s...' % name)
        ! gunzip $source

    # process 
    conn = sqlite3.connect(source.replace('.gz', ''))
    matched_sentences, split_sentences = get_matched_sentences(conn)
    matched_sentences['source'] = name
    split_sentences['source'] = name
    # append
    all_matched.append(matched_sentences)
    all_split.append(split_sentences)
    
    print('zipping %s...' % name)
    unzipped_s = source.replace('.gz', '')
    ! gzip $source

  0%|          | 0/7 [00:00<?, ?it/s]

unzipping ap...
zipping ap...
gzip: can't stat: ../data/diffengine-diffs/spark-output/ap-matched-sentences.db.gz (../data/diffengine-diffs/spark-output/ap-matched-sentences.db.gz): No such file or directory
unzipping independent...
zipping independent...
gzip: can't stat: ../data/diffengine-diffs/spark-output/independent-matched-sentences.db.gz (../data/diffengine-diffs/spark-output/independent-matched-sentences.db.gz): No such file or directory
unzipping bbc...
zipping bbc...
gzip: can't stat: ../data/diffengine-diffs/spark-output/bbc-2-matched-sentences.db.gz (../data/diffengine-diffs/spark-output/bbc-2-matched-sentences.db.gz): No such file or directory
unzipping nyt...
gunzip: ../data/diffengine-diffs/spark-output/nyt-matched-sentences.db: unknown suffix -- ignored
zipping nyt...
unzipping reuters...
zipping reuters...
gzip: can't stat: ../data/diffengine-diffs/spark-output/reuters-matched-sentences.db.gz (../data/diffengine-diffs/spark-output/reuters-matched-sentences.db.gz): No s

DatabaseError: Execution failed on sql '
        with c1 as 
            (SELECT entry_id, version, COUNT(1) as c from split_sentences GROUP BY entry_id, version)
        SELECT entry_id, version from c1
        WHERE c < 10 and c > 5
    ': no such table: split_sentences

In [78]:
def get_matched_sentences(conn):
    low_count_versions = pd.read_sql('''
        with c1 as 
            (SELECT entry_id, version, COUNT(1) as c from split_sentences GROUP BY entry_id, version)
        SELECT entry_id, version from c1
        WHERE c < 10 and c > 5
    ''', con=conn)

    # get join keys
    low_count_entry_ids = ', '.join(list(map(str, low_count_versions['entry_id'].unique())))
    joint_keys = low_count_versions.pipe(lambda df: df['entry_id'].astype(str) + '-' + df['version'].astype(str))
    joint_keys = "'%s'" % "', '".join(joint_keys.tolist())

    # matched sentences
    matched_sentences = pd.read_sql('''
        WITH c1 as ( 
        SELECT *, 
        entry_id || '-' || version_x as key_x,
        entry_id || '-' || version_y as key_y 
        FROM matched_sentences 
        )
        SELECT *
        FROM c1
        WHERE key_x in (%s) AND key_y  in (%s)
        ''' % (joint_keys, joint_keys)
    , con=conn)

    # get split sentences
    split_sentences = pd.read_sql('''
        with c1 AS (
            SELECT *, entry_id || '-' || version as key FROM split_sentences
        )
        SELECT * from c1
        WHERE key IN (%s)
    ''' % joint_keys, con=conn)
    
    return matched_sentences, split_sentences